# Image Scaling with Machine Learning
## Inputs:
    1) Number of image pairs
    2) dictionary of {size_label_A: [pixels_high, pixels_wide], 
                      size_label_B: [pixels_high, pixels_wide]}
    3) run directory
    4) output directory
    5) existing sets parameters.json (None for first run)
***
## Parameters to generate a set of images for train-test-validate
    * Over all parameters
        * number of image sets
        * sizes dict {name1: [h, w], name2: [h, w],...}
        * run-limits {escape_boundry: diagonal_multiplier, max_iterations: int}
        * coloring scheme - perhaps a function
        * output file format
        * ouput file naming format
    * File system
        * output directory {train: path, test: path, validate: path}
        * run-log location
    * Stocastic Choices:
        * hashed dictionary - to reject duplicate images
        * equation
            * parameters
        * rotation 
        * center point 
        * frame scale

## Source structure
    * main - opens run_parameters file - passes to module function
    * selection and tracking module
    * starting plain complex
    * equations module
    * iteration module
    * matrix to image module
***
[PIL image](https://pillow.readthedocs.io/en/5.1.x/reference/Image.html#the-image-class) <br>
****
[GitHub scalygraphic](https://github.com/dlanier/scalygraphic/) <br>
### Background Code:
[Add: random plane on domain](https://github.com/dlanier/scalygraphic/blob/master/scalygraphic/zplain.py) <br>

#### Import & Expand *functions_demo.py*
[FMF functions_demo](https://github.com/dlanier/FlyingMachineFractal/blob/master/pyreimpic/functions_demo_01.py) <br>
#### Import & Trim itergators.py
[FMF itergators](https://github.com/dlanier/FlyingMachineFractal/blob/master/src/itergataters.py) <br>
#### Rewrite HSV coloring
[FMF graphic_utility.py](https://github.com/dlanier/FlyingMachineFractal/blob/master/src/graphic_utility.py) <br>
[FMF numcolorpy.py](https://github.com/dlanier/FlyingMachineFractal/blob/master/src/numcolorpy.py) <br>
#### Reslove plain functions
[FMF z_plane.py](https://github.com/dlanier/FlyingMachineFractal/blob/master/src/z_plane.py) <br>
****
    
### Inception: Random Selection, Tracking and Generation - called from *main()*

In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import os
import sys
import time

sys.path.insert(0, '../src/')
sys.path.insert(0, 'scalygraphic/src/')

# import zplain as zp
# import eq_iter
# import deg_0_ddeq
# import numcolorpy as ncp
from im_scale_products import *

# Clickety Clack:
## production_parameters: *output_dir_name, image_sizes_dict, number_of_sets*
### get a random eq & p with a random domain
#### check uniquness
### get a random color map
## Iterate: run, write, report

In [ ]:
# %%writefile ../src/im_scale_products.py
"""
Collection of functions to run image production for machine learning applications

See the Makefile and ../data/run_files/ for usage examples
"""
import warnings
warnings.filterwarnings('ignore')

import os
import sys
import time
import hashlib
import inspect
from tempfile import TemporaryDirectory

import numpy as np
import PIL

# development running from clone-mount directory or (this) src dir
sys.path.insert(0, '../src/')
sys.path.insert(0, 'scalygraphic/src/')

import zplain as zp
import eq_iter
import deg_0_ddeq
# import numcolorpy as ncp
import impute_color as ncp

"""     Constants (lookups):

        EQUS_DICT is an enumerated dictionary of the functions in module deg_0_ddeq
        EQUS_DICT_NAMED_IDX is a dictionary index {name: enumeration_number} of EQUS_DICT
"""
EQUS_DICT = {k: v for k, v in enumerate(inspect.getmembers(deg_0_ddeq, inspect.isfunction))}
EQUS_DICT_NAMED_IDX = {v[0]: k for k, v in EQUS_DICT.items()}

def get_rand_eq_p_set():
    """ get a random equation and parameter set from the deg_0_ddeq module
    (No Args:)
    
    Returns:
        tuple:      (function_name, function_handle, parameter_set)
    """
    n = np.random.randint(0,len(EQUS_DICT),1)
    fcn_name, fcn = EQUS_DICT[n[0]]
    p = fcn(0.0, None)

    return (fcn_name, fcn, p)

def get_eq_by_name(fcn_name):
    """ get the function handle from the function name
    
    Args:
        fcn_name:   name of a function in deg_0_ddeq
        
    Returns:
        fcn_handle: callable function Z = fcn_name(Z, p, (Z0), (ET))
        
    """
    if fcn_name in EQUS_DICT_NAMED_IDX:
        return EQUS_DICT[EQUS_DICT_NAMED_IDX[fcn_name]][1]
    else:
        return None
    

def get_random_domain(bounds_dict=None):
    """ Usage: 
    domain_dict = get_random_domain(h, w, bounds_dict)
    
    Args:
        bounds_dict:    min - max limits for keys eg.
                            CP_magnitude_limits = {'min': 0, 'max': 7}
                            ZM_limits           = {'min': np.finfo(float).eps, 'max': 2}
                            theta_limits        = {'min': 0, 'max':2 * np.pi}
                        
    Returns:
        domain_dict:    with keys:
                            center_point
                            zoom
                            theta
    """
    domain_dict = {}
    if bounds_dict is None:
        CP_magnitude_limits =   {'min': 0, 'max': 2}
        ZM_limits =             {'min': np.finfo(float).eps, 'max': 1}
        theta_limits =          {'min': 0, 'max':2 * np.pi}
    else:
        CP_magnitude_limits =   bounds_dict['CP_magnitude_limits']
        ZM_limits =             bounds_dict['ZM_limits']
        theta_limits =          bounds_dict['theta_limits']

    r = np.random.uniform(low=0.0, high=2*np.pi) * 0.0+1.0j
    m = np.random.uniform(low=CP_magnitude_limits['min'], high=CP_magnitude_limits['max'])
    domain_dict['center_point'] = m*np.exp(r)
    domain_dict['zoom'] = np.random.uniform(low=ZM_limits['min'], high=ZM_limits['max'])
    domain_dict['theta'] = np.random.uniform(low=theta_limits['min'], high=theta_limits['max'])
    
    return domain_dict

def sha256sum(s):
    """ convert a string to a 256 bit hash key as a string
    
    Args:
        s:          string
        
    Returns:
        hash_key:   256 bit hex as string

    """
    h  = hashlib.sha256()
    h.update(bytes(s, 'ascii'))
    
    return h.hexdigest()

def hash_parameters(domain_dict, fcn_name, p):
    """ get a hash value of equation production parameters to compare uniqueness
    
    Args:
        domain_dict:    parameters defining numerical domain on the complex plain
        fcn_name:       equation function name
        p:              equation parameter inputs
        
    Returns:
        hash_key:   256 bit hex as string
    """
    N_DEC = 15
    f = zp.get_frame_from_dict(domain_dict)
    s = zp.complex_frame_dict_to_string(f, N_DEC) + '\n' + fcn_name
    if isinstance(p, list):
        p_str = ''
        for p_n in p:
            p_str += zp.complex_to_string(p_n, N_DEC)
    else:
        p_str = zp.complex_to_string(p, N_DEC)
    
    s += p_str
    
    return sha256sum(s)

def get_im(ET, Z, Z0):
    """ get a color image from  the products of the escape-time-algorithm Using HSV - RGB model:
    ETn         normalized escape time matrix           Hue
    Zr          normalized rotation of |Z - Z0|         Saturation
    Zd          normalized magnitude of |Z - Z0|        Value
    
    Args:
        ET:     (Integer) matrix of the Escape Times    
        Z:      (complex) matrix of the final vectors   
        Z0:     (complex) matrix of the starting plane
        
    Returns:
        I:      RGB PIL image

    """
    n_rows = np.shape(ET)[0]
    n_cols = np.shape(ET)[1]
    Zd, Zr, ETn = ncp.etg_norm(Z0, Z, ET)

    A = np.zeros((n_rows, n_cols, 3))
    A[:,:,0] += ETn     # Hue
    A[:,:,1] += Zr      # Saturation
    A[:,:,2] += Zd      # Value
    I = PIL.Image.fromarray(np.uint8(A * 255), 'HSV').convert('RGB')
    
    return I

def get_gray_im(ET, Z, Z0):
    """ get a gray-scale image from the products of the escape-time-algorithm
    
    Args:
        ET:     (Integer) matrix of the Escape Times
        Z:      (complex) matrix of the final vectors
        Z0:     (complex) matrix of the starting plane
        
    Returns:
        I:      grayscale PIL image
    """
    return get_im(Z0, Z, ET).convert('L')

def now_name(prefi_str=None, suffi_str=None):
    """ get a human readable time stamp name 
    
    Args:
        prefi_str:
        suffi_str:
        
    Returns:
        now_string: prefi_str + '_' + formatted_time_string + suffi_str
                    eg.  myfilebasename_Mon_23_Sep_2019_06_06_54.tiff
    """
    t0 = time.time()
    t_dec = t0 - np.floor(t0)
    ahora_nombre = time.strftime("%a_%d_%b_%Y_%H_%M_%S", time.localtime()) 
    if prefi_str is None: prefi_str = ''
    if suffi_str is None: suffi_str = ''
        
    return prefi_str + '_' + ahora_nombre + suffi_str


def write_n_image_sets(n_2_do, iteration_dict, small_scale, large_scale, output_dir, hash_list=[]):
    """
    Args:
        n_2_do:         number of pairs of images
        iteration_dict: {'it_max': 64, 'scale_dist': 10} escape-time-algorithm iteration limits
        small_scale:    [h, w] for smaller copy of image
        large_scale:    [h, w] for larger copy of image
        output_dir:     directory will be written if DNE
        hash_list       empty list []  OR  list returned by this function - to avoid duplicates
        
    Returns:
        hash_list:      list of parameter encodings to insure uniqueness 
                        if producing large dataset over multiple sessions or locations
                        
    Writes n_2_do image pairs like:
                        asdregshaaldfkaproiapw_small.jpg
                        asdregshaaldfkaproiapw_large.jpg
    """
    if os.path.isdir(output_dir) == False:
        os.makedirs(output_dir)
                
    print(now_name('Write %i image pairs to \n%s\nStart '%(n_2_do, output_dir)))
    
    if len(hash_list) > 0:
        print('checking duplicates using input hash_list size = %i'%(len(hash_list)))
    else:
        print('new hash list started')

    with TemporaryDirectory() as test_temporary_dir:

        for k_do in range(n_2_do):
            fcn_name, eq, p = get_rand_eq_p_set()
            domain_dict = get_random_domain()

            domain_dict['it_max'] = iteration_dict['it_max']
            domain_dict['max_d'] = iteration_dict['scale_dist'] / domain_dict['zoom']

            hash_idx = hash_parameters(domain_dict, fcn_name, p)
            if hash_idx in hash_list:
                pass
            else:
                hash_list.append(hash_idx)
                domain_dict['n_rows'] = small_scale[0]
                domain_dict['n_cols'] = small_scale[1]
                domain_dict['dir_path'] = test_temporary_dir

                list_tuple = [(eq, (p))]

                t0 = time.time()
                ET, Z, Z0 = eq_iter.get_primitives(list_tuple, domain_dict)
                if GRAYSCALE == True:
                    I = get_gray_im(ET, Z, Z0)
                else:
                    I = get_im(ET, Z, Z0)

                file_name = os.path.join(output_dir, hash_idx + '_' + 'small.jpg')
                I.save(file_name)

                domain_dict['n_rows'] = large_scale[0]
                domain_dict['n_cols'] = large_scale[1]

                ET, Z, Z0 = eq_iter.get_primitives(list_tuple, domain_dict)
                if GRAYSCALE == True:
                    I = get_gray_im(ET, Z, Z0)
                else:
                    I = get_im(ET, Z, Z0)

                file_name = os.path.join(output_dir, hash_idx + '_' + 'large.jpg')
                I.save(file_name)
                print('\n%3i of %3i) %s\t\t'%(k_do+1, n_2_do, fcn_name), 
                      '%0.3f seconds (large & small image written)\n'%(time.time() - t0), 
                      hash_idx)
                
    print('\n', now_name('%i pairs written,\nFinished '%(k_do + 1)))
    
    return hash_list


# Demos:

****
## get a random function and parameter set

In [ ]:
n_trys = 5

for k in range(n_trys):
    fcn_name, eq, p = get_rand_eq_p_set()
    
    print('\n\t', fcn_name)
    try:
        Z = 0.0+0.0j
        print(Z)
        print(eq(Z,p))
        Z = 1.0+1.0j
        print(Z)
        print(eq(Z,p))
    except:
        print('Crash crash ')
        break
        pass

## get a random complex domain

In [ ]:
N_DEC=4

domain_dict = get_random_domain()
f = zp.get_frame_from_dict(domain_dict)
s = zp.complex_frame_dict_to_string(f, N_DEC)
print(s,'\n%50s'%('(from dictionary)'))
for k, v in domain_dict.items():
    if isinstance(v, complex):
        v_str = zp.complex_to_string(v)
    else:
        v_str = '%i'%(v)
        
    print('%40s: %s'%(k, v_str))

## *hash_parameters( )* - function, parameters & domain - (reject duplicate parameters)
    Option: hashed dict > json file to allow reproduction of an exact data-set with different scaling

In [ ]:
fcn_name, eq, p = get_rand_eq_p_set()
domain_dict = get_random_domain()

hash_key = hash_parameters(domain_dict, fcn_name, p)
print(hash_key)

## Produce many pair of scaled images

In [ ]:
#             or use saved list
hash_list=[]

In [ ]:
cell_start_time = time.time()

n_2_do = 2
iteration_dict = {'it_max': 64, 'scale_dist': 10}
small_scale = [128, 128]
large_scale = [255, 255]

output_directory = '../../ImagesSunday'
if os.path.isdir(output_directory) == False:
    os.makedirs(output_directory)

hash_list = write_n_image_sets(n_2_do, iteration_dict, small_scale, large_scale, output_directory, hash_list)

tt = time.time() - cell_start_time
print('\ntotal cell time: %0.2f seconds'%(tt))

## file nameing ?

In [ ]:
do_da = now_name(prefi_str=fcn_name, suffi_str='.jpg')
print(do_da)
d0_dat = hash_list[0] + '.jpg'
print(d0_dat)

## coloring logic
    * HSV model - choose three result components
        * Hue        (red to red) (0, 1) * 255          == visible spectrum colors
        * Saturation (gray to solid color) (0, 1) * 255 == color intensity
        * Value      (black to white) (0, 1) * 255      == brightness
    * RGB conversion from HSV
        * portable for multiple file formats
        * norm for train - validate - test
    * Escape-time algorithm produces
        * Escape Time (integer matrix)
        * Final Vector (Z - Z0)
            * distance (float matrix)
            * rotation (float matrix)
****
### Coloring component normalization:


In [ ]:
def view_component_color_range(Z0, Z, ET):
    # Examine color ranges - for HSV - RGB conversion 
    Zd_t, Zr_t, ETn_t = ncp.etg_norm(Z0, Z, ET)
    print('\nZd_t', np.max(Zd_t), np.min(Zd_t))
    print('Zr_t', np.max(Zr_t), np.min(Zr_t))
    print('ETn_t', np.max(ETn_t), np.min(ETn_t))

## Development legacy - cell below replaced by function:
```python
# Use hash_list for multi-session calling to avoid (improbable) duplicate parameter sets
hash_list = write_n_image_sets(n_2_do, iteration_dict, small_scale, large_scale, output_directory, hash_list)
```

In [ ]:
hashy_list = []

In [ ]:
cell_start_time = time.time()

n_2_do = 3

test_temporary_dir = '../../test_temporary_dir'
if os.path.isdir(test_temporary_dir) == False:
    os.makedirs(test_temporary_dir)

DISPLAY_IN_NOTEBOOK = True
GRAYSCALE = True

small_scale = [128, 128]
large_scale = [255, 255]
output_directory = '../../ImagesFriday'
if os.path.isdir(output_directory) == False:
    os.makedirs(output_directory)
    

for k_do in range(n_2_do):
    fcn_name, eq, p = get_rand_eq_p_set()
    print('\n%3i) %s'%(k_do+1, fcn_name))
    domain_dict = get_random_domain()

    domain_dict['it_max'] = 64
    domain_dict['max_d'] = 10 / domain_dict['zoom']

    hash_idx = hash_parameters(domain_dict, fcn_name, p)
    if hash_idx in hashy_list:
        print('\n\n\t\tImpossible! But! Skipping:\n%s\n'%(hash_idx))
    else:
        hashy_list.append(hash_idx)
        print(hash_idx + '\n')
        domain_dict['n_rows'] = small_scale[0]
        domain_dict['n_cols'] = small_scale[1]

        domain_dict['dir_path'] = test_temporary_dir

        list_tuple = [(eq, (p))]

        t0 = time.time()
        ET, Z, Z0 = eq_iter.get_primitives(list_tuple, domain_dict)
        if GRAYSCALE == True:
            I = get_gray_im(ET, Z, Z0)
        else:
            I = get_im(ET, Z, Z0)
            
        file_name = os.path.join(output_directory, hash_idx + 'small.jpg')
        I.save(file_name)
        if DISPLAY_IN_NOTEBOOK:
            display(I)

        domain_dict['n_rows'] = large_scale[0]
        domain_dict['n_cols'] = large_scale[1]

        ET, Z, Z0 = eq_iter.get_primitives(list_tuple, domain_dict)
        if GRAYSCALE == True:
            I = get_gray_im(ET, Z, Z0)
        else:
            I = get_im(ET, Z, Z0)
            
        file_name = os.path.join(output_directory, hash_idx + 'large.jpg')
        I.save(file_name)
        
        print('%0.3f\t 2 images time'%(time.time() - t0))
        if DISPLAY_IN_NOTEBOOK:
            display(I)

tt = time.time() - cell_start_time
print('\n%i pairs written in %0.2f seconds'%(n_2_do, tt))

## coloring logic
    * HSV model - choose three result components
        * Hue        (red to red) (0, 1) * 255          == visible spectrum colors
        * Saturation (gray to solid color) (0, 1) * 255 == color intensity
        * Value      (black to white) (0, 1) * 255      == brightness
    * RGB conversion from HSV
        * portable for multiple file formats
        * norm for train - validate - test
    * Escape-time algorithm produces
        * Escape Time (integer matrix)
        * Final Vector (Z - Z0)
            * distance (float matrix)
            * rotation (float matrix)
****
### Coloring component normalization:


In [ ]:
def view_component_color_range(Z0, Z, ET):
    # Examine color ranges - for HSV - RGB conversion 
    Zd_t, Zr_t, ETn_t = ncp.etg_norm(Z0, Z, ET)
    print('\nZd_t', np.max(Zd_t), np.min(Zd_t))
    print('Zr_t', np.max(Zr_t), np.min(Zr_t))
    print('ETn_t', np.max(ETn_t), np.min(ETn_t))
    
view_component_color_range(Z0, Z, ET)

## file nameing calls - time-stamp-name or hash-name

In [ ]:
do_da = now_name(prefi_str=fcn_name, suffi_str='.jpg')
print(do_da)
d0_dat = hashy_list[0] + '.jpg'
print(d0_dat)